# Pure ResNet train on cifar10

`$ sudo rmmod nvidia_uvm`  
`$ sudo modprobe nvidia_uvm`

In [24]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchvision
from torchvision.transforms import transforms
from torchvision import models

In [25]:
# check if gpu training is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
else:
    device = torch.device('cpu')
    
print("Using...", device)

Using... cuda


In [26]:
model = models.resnet50(pretrained=False, num_classes=10)
model.to(device)

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [15]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 256
n_epoch=400

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

checkpoint_name="resnet_ch3"

Files already downloaded and verified
Files already downloaded and verified


In [16]:
from atm.simclr.utils import save_checkpoint
import os

In [17]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [21]:
from torch.utils.tensorboard import SummaryWriter
tb = SummaryWriter()

def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()


In [23]:
for epoch in range(n_epoch):  # loop over the dataset multiple times
    running_loss = 0.0
    total_correct = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        preds = model(inputs)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        total_correct+= get_num_correct(preds, labels)
        
    
    tb.add_scalar("Loss", running_loss, epoch)
    tb.add_scalar("Correct", total_correct, epoch)
    tb.add_scalar("Accuracy", total_correct/ len(trainset), epoch)
    
print('Finished Training')

Finished Training


In [19]:
save_checkpoint({
        'epoch': n_epoch,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        }, is_best=False, filename=os.path.join('./', f"resnet_ch3_{epoch}.pth"))

In [8]:
loss

tensor(0.0224, device='cuda:0', grad_fn=<NllLossBackward>)

In [7]:
import argparse 

args = argparse.Namespace()

args.data='./datasets' 
args.dataset_name='cifar10'
args.arch='resnet50'
args.workers=1
args.epochs=300 
args.batch_size=256 
args.lr=0.02
args.wd=0.0005
args.disable_cuda=False
args.fp16_precision=True
args.out_dim=128
args.log_every_n_steps=100
args.temperature=0.07
args.n_views = 2
args.gpu_index=0
args.device='cuda' if torch.cuda.is_available() else 'cpu'

print("Using device:", args.device)


assert args.n_views == 2, "Only two view training is supported. Please use --n-views 2."
# check if gpu training is available
if not args.disable_cuda and torch.cuda.is_available():
    args.device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
else:
    args.device = torch.device('cpu')
    args.gpu_index = -1

Using device: cuda


In [3]:
import pickle
import numpy as np

ddir = "../../tonemap/bf_data/Nair_and_Abraham_2010/"

fn = ddir + "all_gals.pickle"
all_gals = pickle.load(open(fn, "rb"))

all_gals = all_gals[1:] # Why the first galaxy image is NaN?

good_gids = np.array([gal['img_name'] for gal in all_gals])

from astrobf.utils.misc import load_Nair
cat_data = load_Nair(ddir + "catalog/table2.dat")
# pd dataframe

cat = cat_data[cat_data['ID'].isin(good_gids)]

tmo_params = {'b': 6.0,  'c': 3.96, 'dl': 9.22, 'dh': 2.45}




## DataLoader

In [4]:
import matplotlib.pyplot as plt 

from PIL import Image
from typing import Any, Callable, Optional, Tuple
from torchvision.datasets.vision import VisionDataset
from functools import partial
from astrobf.tmo import Mantiuk_Seidel

class TonemapImageDataset(VisionDataset):
    def __init__(self, 
                 data_array, 
                 tmo,
                 labels: Optional = None, 
                 train: bool=True,
                 transform: Optional[Callable] = None,
                 target_transform: Optional[Callable] = None,):
        self._array = data_array
        self._good_gids = np.array([gal['img_name'] for gal in data_array])
        self.img_labels = labels
        self.transform = transform
        self.target_transform = target_transform
        self.tmo = tmo
        self._bad_tmo=False

    def _apply_tm(self, image):
        try:
            return self.tmo(image)
        except ZeroDivisionError:
            print("division by zero. Probably bad choice of TM parameters")
            self._bad_tmo=True
            return image

    def _to_8bit(self, image):
        """
        Normalize per image (or use global min max??)
        """

        image = (image - image.min())/image.ptp()
        image *= 255
        return image.astype('uint8')        
    
    def __len__(self) -> int:
        return len(self._array)
    
    def __getitem__(self, idx: int) -> Tuple[Any, Any]:
        """
        For super
        """
        image, _segmap, weight = self._array[idx]['data']
        image[~_segmap.astype(bool)] = 0#np.nan # Is it OK to have nan?
        image[image < 0] = 0

        image = self._to_8bit(self._apply_tm(image))
        image = Image.fromarray(image)
        label = self.img_labels[idx]
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label


In [ ]:
from torch.utils.data import SubsetRandomSampler

args.batch_size = 512
# data prepare
frac_train = 0.8

all_data = TonemapImageDatasetPair(all_gals, partial(Mantiuk_Seidel, **tmo_params),
                                     labels=cat['TT'].to_numpy(),
                                     train=True, 
                                     transform=train_transform)

all_data_val = TonemapImageDataset(all_gals, partial(Mantiuk_Seidel, **tmo_params),
                                     labels=cat['TT'].to_numpy(),
                                     train=False, 
                                     transform=test_transform)
len_data = len(all_data)

data_idx = np.arange(len_data)
np.random.shuffle(data_idx)
ind = int(np.floor(len_data * frac_train))
train_idx, test_idx = data_idx[:ind], data_idx[ind:]

train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)

train_loader = DataLoader(all_data, batch_size=args.batch_size, shuffle=False, num_workers=16, pin_memory=True, drop_last=True,
                         sampler=train_sampler)

test_loader = DataLoader(all_data_val, batch_size=args.batch_size, shuffle=False, num_workers=16, pin_memory=True, drop_last=True,
                         sampler=test_sampler)
